In [208]:
import pandas as pd
from collections import Counter 
from helper import load_data,pipeline, unique_drugs, generate_drugs_df
import re


%pylab inline 
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [209]:
trials = load_data()
c = unique_drugs(trials)
df, exceptions = generate_drugs_df(c, trials)
df

/Users/nmp256/Code/datascience/metis_projects/breast cancer/helper.py:168: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  pass_phase_1,


got 1001 results and had 0 exceptions
probs are[22.78, 29.82, 52.94]


,drug,num_of_trials,phase_1_success,phase_2_success,phase_3_success,phase_1_2_success,phase_2_3_success,phase_1_start,phase_1_first_end,phase_1_last_end,phase_2_start,phase_2_first_end,phase_2_last_end
26,Trastuzumab,441,True,True,True,True,True,2002-04-01,2002-05-01,2019-03-01,1998-04-01,1999-09-01,2019-04-01
5,Paclitaxel,269,True,True,True,True,True,2005-12-01,2006-01-25,2019-01-24,1998-04-01,1999-11-01,2019-01-25
111,Lapatinib,181,True,True,True,True,True,2003-04-28,2003-11-01,2016-01-06,2002-10-01,2003-12-09,2018-05-01
39,Capecitabine,175,True,True,True,True,True,2000-08-01,2001-11-01,2019-01-31,1998-07-01,2001-09-01,2019-03-15
4,Bevacizumab,167,True,True,True,True,False,2005-09-01,2005-10-01,2015-11-23,2001-11-01,2001-12-01,2018-07-01
46,Eribulin,149,True,True,True,True,False,2011-10-01,2011-11-01,2018-05-30,2004-09-01,2006-09-20,2019-01-01
21,Palbociclib,140,True,True,True,True,True,2011-03-01,2012-02-01,2019-03-01,2009-10-01,2013-02-22,2019-03-01
124,Docetaxel,136,True,True,True,True,True,1996-12-01,1997-01-01,2011-01-01,1997-01-01,1997-06-01,2019-03-01
32,Letrozole,132,True,True,True,True,True,2010-03-01,2010-09-01,2018-12-11,2000-01-01,2000-07-26,2019-04-01
53,Everolimus,125,True,True,True,True,False,2005-11-01,2006-08-16,2017-07-17,2005-01-19,2005-08-03,2018-11-28


In [193]:
df[df['drug']=='Perflubutane']

,drug,num_of_trials,phase_1_success,phase_2_success,phase_3_success,phase_1_2_success,phase_2_3_success,phase_1_start,phase_1_first_end,phase_1_last_end,phase_2_start,phase_2_first_end,phase_2_last_end
458,Perflubutane,1,True,True,False,False,False,2014-10-20,2017-12-01,2014-10-20,2009-01-15,2010-09-13,2017-12-01


In [166]:
def prob_of_success(drug_trials):
    # see if the drug has a trial in each phase.
    unique_trial_values = drug_trials["Phase of Trial"].unique()
    (
        pass_phase_1,
        pass_phase_2,
        pass_phase_3,
        pass_phase_one_two,
        pass_phase_two_three,
    ) = (False, False, False, False, False)
    if ("Phase I" in unique_trial_values) & ("Phase II" in unique_trial_values):
        pass_phase_1 = True

    if ("Phase II" in unique_trial_values) & ("Phase III" in unique_trial_values):
        pass_phase_2 = True

    if ("Phase III" in unique_trial_values) & ("Phase IV" in unique_trial_values):
        pass_phase_3 = True

    if ("Phase I/II" in unique_trial_values) & ("Phase III" in unique_trial_values):
        pass_phase_one_two = True

    if ("Phase II/III" in unique_trial_values) & ("Phase IV" in unique_trial_values):
        pass_phase_two_three = True

    return [
        pass_phase_1,
        pass_phase_2,
        pass_phase_3,
        pass_phase_one_two,
        pass_phase_two_three,
    ]

In [189]:
df = trials[trials['Primary Drugs'].str.contains(
        'Perflubutane', case=False)]
df['trial_start'] = df.apply(start_to_datetime, axis=1)
df['trial_end'] = df.apply(end_to_datetime, axis=1)

# if  numpy.datetime64('NaT') in df[df['Phase of Trial'] == 'Phase I']['trial_start'].values:
#     print ('hi')

# df[df['Phase of Trial'] == 'Phase I']['trial_start'].values[0]
# df[df['Phase of Trial'] == 'Phase I']['Trial Initiation date'].values
prob_of_success(df)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


[True, True, False, False, False]

In [190]:
df

,Trial ID,Trial Title,Mechanism of action,Drug class (therapeutic effect),Drug class (chemical),Drug class (biological),Indication,Organisations,Trial Design,Trial Endpoints,...,Trial History,Diseases treated,Biomarkers-Inclusion,Biomarkers-Exclusion,Biomarkers-Outcome Measure,Biomarkers-Safety,Biomarkers-not specified,Primary Drugs,trial_start,trial_end
1066,700289971,Efficacy and safety of hookwire-guided sentine...,Ultrasonography enhancers,NaN,"Contrast media, Fluorocarbons, Small molecules",NaN,Breast cancer,NaN,"open, prospective",Concordance of the results between Sonazoid po...,...,2018-10-31|Other trial event|Last checked agai...,Breast cancer|diagnosis|,NaN,NaN,NaN,NaN,NaN,Perflubutane,2017-12-01,NaT
2040,700290448,Pharmacokinetics of Sonazoid™ Following Intrav...,Ultrasonography enhancers,NaN,"Contrast media, Fluorocarbons, Small molecules",NaN,"Breast cancer, Liver cancer, Liver disorders","GE Healthcare, GE Healthcare","open, parallel, prospective",Area Under the Concentration Versus Time Curve...,...,2017-11-16|New trial record|New trial record,Breast cancer|treatment|; Liver cancer|treatme...,NaN,NaN,NaN,NaN,NaN,Perflubutane,2014-10-20,2014-11-20
2980,700271780,DD-723 Phase II clinical trial - Dose-response...,Ultrasonography enhancers,NaN,"Contrast media, Fluorocarbons, Small molecules",NaN,Breast cancer,"Daiichi Sankyo Company, Daiichi Sankyo Company","multicentre, prospective, randomised, single-b...",Effectiveness of contrast effect\n\nRight diag...,...,2016-05-25|New trial record|New trial record,Breast cancer|treatment|,NaN,NaN,NaN,NaN,NaN,Perflubutane,2009-01-15,2009-12-31
2981,700271654,DD-723 Phase III Clinical Trial - Confirmatory...,Ultrasonography enhancers,NaN,"Contrast media, Fluorocarbons, Small molecules",NaN,"Breast cancer, Male breast cancer","Daiichi Sankyo Company, Daiichi Sankyo Company","multicentre, open, prospective",Diagnostic accuracy rate of differential diagn...,...,2016-05-24|New trial record|New trial record,Breast cancer|diagnosis|; Male breast cancer|d...,NaN,NaN,NaN,NaN,NaN,Perflubutane,2010-09-13,2012-03-31


In [200]:
messy_drug_list = list(trials["Primary Drugs"].values)
single_word_drugs_list = [drug for row in messy_drug_list for drug in row.split(", ")]
c = Counter(
    np.array([drug for row in single_word_drugs_list for drug in row.split(", ")])
)
filtered_c = {k: v for k, v in c.items() if v > 0}

In [199]:
len(unique_drug_list)

1587

In [183]:
df[df['Phase of Trial'] == 'Phase III']['trial_start'].values[0]

numpy.datetime64('2010-09-13T00:00:00.000000000')

In [163]:
pipeline('Trastuzumab', 120, trials)

['Trastuzumab',
 120,
 True,
 True,
 True,
 True,
 True,
 numpy.datetime64('2002-04-01T00:00:00.000000000'),
 numpy.datetime64('2002-05-01T00:00:00.000000000'),
 numpy.datetime64('2019-03-01T00:00:00.000000000'),
 numpy.datetime64('1998-04-01T00:00:00.000000000'),
 numpy.datetime64('1999-09-01T00:00:00.000000000'),
 numpy.datetime64('2019-04-01T00:00:00.000000000')]